# Week 10

Text Processing and Analysis

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/text_utils.py
!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/datasets/text/movie_reviews.tar.gz | tar xz

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import string

from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from data_utils import MinMaxScaler
from data_utils import classification_error, display_confusion_matrix

from text_utils import get_top_words

## Text Classification

Let's ____ using a movie review dataset.

Load and look.

In [ ]:
reviews_df = pd.read_csv("./data/text/movie_reviews.csv")
reviews_df.head()

### Features

What can we say?

In [ ]:
def count_characters(st):
  return len("".join(st.split()))

def count_words(st):
  return len(st.split(" "))

def count_punctuation(st):
  return len([c for c in st if c in string.punctuation])

def count_digits(st):
  return len([c for c in st if c in string.digits])

def get_punctuation_pct(st):
  return count_punctuation(st) / count_characters(st)

def get_digit_pct(st):
  return count_digits(st) / count_characters(st)

Apply to the `DataFrame`.

In [ ]:
reviews_df["char_count"] = reviews_df["review"].apply(count_characters)
reviews_df["word_count"] = reviews_df["review"].apply(count_words)
reviews_df["punctuation_pct"] = reviews_df["review"].apply(get_punctuation_pct)
reviews_df["digit_pct"] = reviews_df["review"].apply(get_digit_pct)

reviews_df

Look at some of these features

In [ ]:
plt.scatter(reviews_df["word_count"], reviews_df["punctuation_pct"], c=reviews_df["sentiment"])
plt.title("Punctuation % x Word Count")
plt.show()

plt.scatter(reviews_df["digit_pct"], reviews_df["punctuation_pct"], c=reviews_df["sentiment"])
plt.title("Digit % x Word Count")
plt.show()

plt.scatter(reviews_df["word_count"], reviews_df["char_count"], c=reviews_df["sentiment"])
plt.title("Character Count x Word Count")
plt.show()

Scale

In [ ]:
mScaler = MinMaxScaler()

simple_feats_df = reviews_df.drop(columns=["review", "sentiment"])
simple_feats_scaled_df = mScaler.fit_transform(simple_feats_df)

simple_feats_scaled_df["sentiment"] = reviews_df["sentiment"]

simple_feats_scaled_df

In [ ]:
reviews_train_df, reviews_test_df = train_test_split(simple_feats_scaled_df, test_size=0.2)

reviews_train_df

In [ ]:
mClassifier = RandomForestClassifier()

train_feats = reviews_train_df.drop(columns=["sentiment"])
train_labels = reviews_train_df["sentiment"]

mClassifier.fit(train_feats, train_labels)

train_preds = mClassifier.predict(train_feats)

classification_error(train_labels, train_preds)

In [ ]:
test_feats = reviews_test_df.drop(columns=["sentiment"])
test_labels = reviews_test_df["sentiment"]

test_preds = mClassifier.predict(test_feats)

classification_error(test_labels, test_preds)

# 🤔

Bag of words

In [ ]:
reviews_df = pd.read_csv("./data/text/movie_reviews.csv")

reviews_train_df, reviews_test_df = train_test_split(reviews_df, test_size=0.2)
reviews_train_df

In [ ]:
mCV = CountVectorizer(stop_words="english", min_df=5, max_df=0.75, max_features=10_000)

reviews_train_vct = mCV.fit_transform(reviews_train_df["review"])
reviews_test_vct = mCV.transform(reviews_test_df["review"])

In [ ]:
reviews_train_vct

Working with sparse matrices.

Words counted:

In [ ]:
vocab = mCV.get_feature_names_out()
vocab

Get words in a review

In [ ]:
mCV.inverse_transform(reviews_train_vct[0])

Get indices of words in a review:

In [ ]:
reviews_train_vct[0].nonzero()

Get counts of those words in a review

In [ ]:
reviews_train_vct[reviews_train_vct[0].nonzero()]

Get words ordered by frequency:

In [ ]:
review = reviews_train_vct[0]

n_words = len(review.nonzero()[0])

sorted_idxs = (-review.toarray()[0]).argsort()

vocab[sorted_idxs[:n_words]]

In [ ]:
from text_utils import get_top_words

get_top_words(reviews_train_vct[0], vocab, n_words)

In [ ]:
mClassifier = RandomForestClassifier()

train_labels = reviews_train_df["sentiment"]

mClassifier.fit(reviews_train_vct, train_labels)

train_preds = mClassifier.predict(reviews_train_vct)

classification_error(train_labels, train_preds)

In [ ]:
test_labels = reviews_test_df["sentiment"]

test_preds = mClassifier.predict(reviews_test_vct)

classification_error(test_labels, test_preds)

Naive Bayes

Some equations

Bernouli vs gaussian vs categorical vs multinomial

In [ ]:
# here

Order of words ?

In [ ]:
mCV = CountVectorizer(stop_words="english", min_df=5, max_df=0.75, max_features=50_000, ngram_range=(2, 2))

reviews_train_vct = mCV.fit_transform(reviews_train_df["review"])
reviews_test_vct = mCV.transform(reviews_test_df["review"])

In [ ]:
vocab = mCV.get_feature_names_out()
vocab

In [ ]:
mCV.inverse_transform(reviews_train_vct[0])

In [ ]:
get_top_words(reviews_train_vct[0], vocab)

In [ ]:
mClassifier = MultinomialNB()

train_labels = reviews_train_df["sentiment"]

mClassifier.fit(reviews_train_vct, train_labels)

train_preds = mClassifier.predict(reviews_train_vct)

classification_error(train_labels, train_preds)

In [ ]:
test_labels = reviews_test_df["sentiment"]

test_preds = mClassifier.predict(reviews_test_vct)

classification_error(test_labels, test_preds)

Slightly smarter counts

In [ ]:
mTfidV = TfidfVectorizer(stop_words="english", min_df=5, max_df=0.75, max_features=50_000, ngram_range=(1, 1))

reviews_train_vct = mTfidV.fit_transform(reviews_train_df["review"])
reviews_test_vct = mTfidV.transform(reviews_test_df["review"])

In [ ]:
vocab = mTfidV.get_feature_names_out()
vocab

In [ ]:
mTfidV.inverse_transform(reviews_train_vct[0])

In [ ]:
get_top_words(reviews_train_vct[0], vocab, 10)

In [ ]:
mClassifier = MultinomialNB()

train_labels = reviews_train_df["sentiment"]

mClassifier.fit(reviews_train_vct, train_labels)

train_preds = mClassifier.predict(reviews_train_vct)

classification_error(train_labels, train_preds)

In [ ]:
test_labels = reviews_test_df["sentiment"]

test_preds = mClassifier.predict(reviews_test_vct)

classification_error(test_labels, test_preds)

TODO: TFIDF with ngrams ??

Can we extract other info ? Cluster ?

In [ ]:
mClust = KMeans(n_clusters=8)
reviews_train_km = mClust.fit_predict(reviews_train_vct)

In [ ]:
get_top_words(mClust.cluster_centers_, mTfidV.get_feature_names_out(), 8)[0]

Can we do better ?

We're clustering over 40k features .... very sparse space.

In [ ]:
mNmf = NMF(n_components=8)
reviews_train_nmf = mNmf.fit_transform(reviews_train_vct)

In [ ]:
get_top_words(mNmf.components_, mTfidV.get_feature_names_out(), 6)[0]

Classification for other dataset.

Amazon products

In [ ]:
!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/raw/refs/heads/main/datasets/text/amazon_reviews/books.tar.gz | tar xz

Read and drop nan

In [ ]:
reviews_full_df = pd.read_csv("./data/text/amazon_reviews/books.csv")
reviews_full_df.dropna(inplace=True)
reviews_full_df

Look at class distribution

In [ ]:
reviews_full_df["rating"].value_counts()

# 🫤

A model can guess $5$ all the time and be correct $60\%$ of the time.

In [ ]:
min_count = reviews_full_df["rating"].value_counts().min()

def sample_min(df):
  return df.sample(min_count)

rg = reviews_full_df.groupby("rating")
reviews_balanced_df = rg.apply(sample_min).reset_index(drop=True)

del reviews_full_df

reviews_balanced_df

Two more prep steps:

In [ ]:
reviews_df = pd.DataFrame(reviews_balanced_df["rating"].astype(int))
reviews_df["review"] = reviews_balanced_df["title"] + " " + reviews_balanced_df["review_text"]
reviews_df

Ok. Go forth and Classify

In [ ]:
# T/T Split
# Vectorize
# Classify
# Evaluate
# Repeat with ngrams